# RFM (vaex)

HTML tag escapes for Vaex index column in jupyterlab https://github.com/vaexio/vaex/issues/656 

In [1]:
from datetime import datetime
import numpy as np
import vaex
import pandas as pd

Leemos el parquet.

In [2]:
df=vaex.from_csv('2019-Oct-Nov-transformed.csv', parse_dates=['event_time'])
df

#,event_time,event_type,product_id,category_code,brand,price,user_id
0,2019-10-01 00:00:00.000000000,purchase,1004856,electronics.smartphone,samsung,130.76,543272936
1,2019-10-01 00:00:00.000000000,purchase,1002532,electronics.smartphone,apple,642.69,551377651
2,2019-10-01 00:00:00.000000000,purchase,5100816,no category,xiaomi,29.51,514591159
3,2019-10-01 00:00:00.000000000,purchase,13800054,furniture.bathroom.toilet,santeri,54.42,555332717
4,2019-10-01 00:00:00.000000000,purchase,4804055,electronics.audio.headphone,apple,189.91,524601178
...,...,...,...,...,...,...,...
"1,659,783",2019-11-30 00:00:00.000000000,purchase,1004874,electronics.smartphone,samsung,346.7,547804983
"1,659,784",2019-11-30 00:00:00.000000000,purchase,1005130,electronics.smartphone,apple,1437.02,515582054
"1,659,785",2019-11-30 00:00:00.000000000,purchase,1004767,electronics.smartphone,samsung,235.6,579876821
"1,659,786",2019-11-30 00:00:00.000000000,purchase,3701309,appliances.environment.vacuum,polaris,89.32,543733099


In [3]:
df['event_time'] = df[('event_time')].astype("datetime64[D]")
df

#,event_time,event_type,product_id,category_code,brand,price,user_id
0,2019-10-01,purchase,1004856,electronics.smartphone,samsung,130.76,543272936
1,2019-10-01,purchase,1002532,electronics.smartphone,apple,642.69,551377651
2,2019-10-01,purchase,5100816,no category,xiaomi,29.51,514591159
3,2019-10-01,purchase,13800054,furniture.bathroom.toilet,santeri,54.42,555332717
4,2019-10-01,purchase,4804055,electronics.audio.headphone,apple,189.91,524601178
...,...,...,...,...,...,...,...
"1,659,783",2019-11-30,purchase,1004874,electronics.smartphone,samsung,346.7,547804983
"1,659,784",2019-11-30,purchase,1005130,electronics.smartphone,apple,1437.02,515582054
"1,659,785",2019-11-30,purchase,1004767,electronics.smartphone,samsung,235.6,579876821
"1,659,786",2019-11-30,purchase,3701309,appliances.environment.vacuum,polaris,89.32,543733099


### RECENCY

In [4]:
window= np.datetime64('2019-12-01')
window = window.astype("datetime64[D]")

print(window)

2019-12-01


In [5]:
df_recency = df.groupby(by=['user_id'], agg=({'event_time': 'max'}))
#df_recency['event_time'] = df_recency[('event_time')].values.astype("datetime64[D]")
df_recency['event_time'] = df_recency['event_time'].values.astype("datetime64[D]")


df_recency

#,user_id,event_time
0,555160488,2019-11-12
1,535252624,2019-10-01
2,555462711,2019-10-01
3,550607818,2019-11-29
4,555463605,2019-10-01
...,...,...
"697,465",541557351,2019-11-25
"697,466",516614592,2019-11-25
"697,467",575799382,2019-11-25
"697,468",521470682,2019-11-25


In [6]:
#df_recency['Recency'] = df_recency['event_time'].apply(lambda x: (window - x))
df_recency['Recency'] =  (df_recency['event_time'] - window) * (-1)
df_recency

#,user_id,event_time,Recency
0,555160488,2019-11-12,19 days +0:00:00
1,535252624,2019-10-01,61 days +0:00:00
2,555462711,2019-10-01,61 days +0:00:00
3,550607818,2019-11-29,2 days +0:00:00
4,555463605,2019-10-01,61 days +0:00:00
...,...,...,...
"697,465",541557351,2019-11-25,6 days +0:00:00
"697,466",516614592,2019-11-25,6 days +0:00:00
"697,467",575799382,2019-11-25,6 days +0:00:00
"697,468",521470682,2019-11-25,6 days +0:00:00


In [7]:
df_recency = df_recency[['user_id','Recency']]

In [8]:
df_recency['Recency'] = df_recency[('Recency')].values.astype("int")
df_recency

#,user_id,Recency
0,555160488,19
1,535252624,61
2,555462711,61
3,550607818,2
4,555463605,61
...,...,...
"697,465",541557351,6
"697,466",516614592,6
"697,467",575799382,6
"697,468",521470682,6


### FREQUENCY

In [ ]:
df_frequency = df.groupby(by=['user_id'], agg='count')
df_frequency

In [ ]:
df_frequency.rename('count','Frequency')
df_frequency

### MONETARY

In [ ]:
df_monetary = df.groupby(by=['user_id'], agg=({'price': 'sum'}))
df_monetary

In [ ]:
df_monetary.rename('price','Monetary')
df_monetary

### MERGE LOS TRES DF

In [ ]:
df_rfm = df_recency.join(df_frequency, 
                        how='inner', 
                        left_on ='user_id',
                        right_on='user_id')
                        
df_rfm

In [ ]:
df_rfm = df_rfm.join(df_monetary, 
                        how='inner', 
                        left_on ='user_id',
                        right_on='user_id')
df_rfm

### Eliminamos outliers utilizando el percentil 98.
Para correr la función percentile_approx tuve que degradar la versión de numpy (1.21.5 a 1.19.5)
https://github.com/vaexio/vaex/issues/385

In [ ]:
#Chequeamos máximos y mínimos en Recency (no puede ser menor a 1 ni mayor a 61)
print(df_rfm['Recency'].min())
print(df_rfm['Recency'].max())

In [ ]:
#Chequeamos máximos y mínimos en Frequency
print(df_rfm['Frequency'].min())
print(df_rfm['Frequency'].max())

In [ ]:
#Chequeamos máximos y mínimos en Monetary
print(df_rfm['Monetary'].min())
print(df_rfm['Monetary'].max())

In [ ]:
#Para Frequency, calculamos el percentil 98 (q98) y nos quedamos con los valores menores a él.

p98 = df_rfm.percentile_approx('Frequency', 98)
print('Percentil 98: ' + str(p98))

df_rfm = df_rfm[df_rfm.Frequency <= p98]
df_rfm

In [ ]:
#Para Monetary, calculamos el percentil 2 y el percentil 98 (p2 y p98) y nos quedamos con el intervalo entre estos dos valores.

p2 = df_rfm.percentile_approx('Monetary', 2)
p98 = df_rfm.percentile_approx('Monetary', 98)

print('Percentil 2: ' + str(p2))
print('Percentil 98: ' + str(p98))

df_rfm = df_rfm[df_rfm.Monetary > p2]
df_rfm = df_rfm[df_rfm.Monetary < p98]
df_rfm


### Asignamos quintiles
[FEATURE-REQUEST] Like Pandas cut https://github.com/vaexio/vaex/issues/1228

Al no encontrar ninguna función similar ni poder aplicar np.select, convierto el DataframeArray a Pandas.

In [ ]:
rfm = pd.DataFrame(df_rfm, columns=['user_id','Recency','Frequency','Monetary'])
rfm = rfm.astype({'user_id':'int','Recency':'int','Frequency':'int'})
rfm

In [ ]:
R_condition = [(rfm['Recency'] < 11),
            ((rfm['Recency'] >= 11) & (rfm['Recency'] < 15)),
            ((rfm['Recency'] >= 15) & (rfm['Recency'] < 28)),
            ((rfm['Recency'] >= 28) & (rfm['Recency'] < 44)),
            (rfm['Recency'] >= 44)]

option = [5,4,3,2,1]
rfm['R'] = np.select(R_condition, option,default=5)


In [ ]:
condition = [(rfm.Frequency == 1) | (rfm.Frequency == 2),
            (rfm.Frequency == 3) | (rfm.Frequency == 4),
            (rfm.Frequency == 5) | (rfm.Frequency == 6),
            (rfm.Frequency == 7) | (rfm.Frequency == 8)]
option = [1,2,3,4]
rfm['F'] = np.select(condition, option, default= 5)

rfm['M'] = pd.qcut(rfm['Monetary'], q=5, labels=[1,2,3,4,5]) 
rfm['M']=rfm['M'].astype(int)
rfm

In [ ]:
#Convertimos de nuevo a un DataframeArrays
rfm = vaex.from_pandas(rfm)
rfm

In [ ]:
rfm["RFM_SCORE"] = (rfm['R'].astype(str) + 
                    rfm['F'].astype(str) +
                    rfm['M'].astype(str))
rfm

### A partir del RFM_SCORE, mapeamos los segmentos.

In [ ]:
seg_map = vaex.read_csv('segmentos_rfm.csv', usecols=['RFM_SCORE','segment'])
seg_map['RFM_SCORE']= seg_map.RFM_SCORE.astype(str)
seg_map.head()

In [ ]:
rfm = rfm.join(seg_map, 
                        how='inner', 
                        left_on ='RFM_SCORE',
                        right_on='RFM_SCORE')
rfm

### How to read this table?
Champions
There are 90063 people in this group.
They do not shop for an average of 4.93 days.
They shopped an average of 4.67 times.
They earned an average of 1753.93 units of money.

In [ ]:
rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean", "count","max"])

In [ ]:
ctas_x_seg=(
    rfm
    .groupby(['segment'],as_index=False)['user_id']
    .count()
)

ctas_x_seg['text_']=ctas_x_seg['segment']+' \n['+ctas_x_seg['user_id'].astype(str)+' ] '
ctas_x_seg.loc[ctas_x_seg['user_id']<100,'user_id']=ctas_x_seg.loc[ctas_x_seg['user_id']<100,'user_id']+1000


plt.figure(figsize=(20,10))
squarify.plot(sizes=list(ctas_x_seg.iduser), 
              label=list(ctas_x_seg.text_), 
              alpha=.4,
              edgecolor="black", linewidth=2,text_kwargs={'fontsize':15})